In [1]:
import dmd_with_power_control_single_task as DMD
import imageio
import numpy as np
import time
from glob import glob
import os
import importlib
importlib.reload(DMD)
import napari as nap

In [2]:
XP_FOLDER = r'D:\Data\OR83b_ppk72__20xpair_trace_subregion__pun_control' # 
TARGET_FOLDER = XP_FOLDER + r'\22_1__mVenus_20241008_150149' # D:\Data\OR83b_ppk72__20xpair_trace_subregion_new\2_1__mVenus_20240812_130150
GENERAL_PARAMETERS = r'C:\\Users\\Light Sheet User\\Desktop\\DMD_parameters.pickled'

mirrors = DMD.dmd(XP_FOLDER, TARGET_FOLDER, GENERAL_PARAMETERS)
# Prints basic information about the DMD.
mirrors.idle()

Loading library: C:/Program Files/ALP-4.3/ALP-4.3 API/x64/alp4395.dll
DMD found, resolution = 1920 x 1200.
DMD patterns folder already exists.


c:\Users\Light Sheet User\Documents\GitHub\DMD\dmd_with_power_control_single_task.py:43: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\\\Users\\\\Light Sheet User\\\\Desktop\\\\DMD_parameters.pickled'>
  self.parameters = pickle.load(open(general_parameters_folder, 'rb'))


SELECT ROIs

In [3]:
mirrors.select_ROIs()

IMPORT the ROIs for USAGE

In [4]:
mirrors.import_N_rois(exp='22_1')

Found 3 ROI(s)


In [3]:
mirrors.idle()

Learning Protocol - Trace Conditioning - 20 Training Trials, 15s long odor stimulus, 1s ISI Gap, 15s ITI gap

In [5]:
masks_path = r'D:\Data\OR83b_ppk72__20xpair_trace_subregion_new\excitation_patterns\26_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\26_1__DAN_20240906_144519' # D:\Data\g12\
protocol_name = r"/26_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

for i in range(2):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(19):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(rep_dead_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(3):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         ], \
                durations=[odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, 60, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, 60, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           ]
                ) # 30.8 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_23700\218344891.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_23700\218344891.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_23700\218344891.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (an

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input

Trace Conditioning Control 1 (Odors Only) - 20 Training Trials, 15s long odor stimulus, 1s ISI Gap, 15s ITI gap

In [ ]:
masks_path = r'D:\Data\OR83b_ppk72__20xpair_trace_subregion__odor_control\excitation_patterns\24_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\24_1__DAN_20240924_173750' # D:\Data\g12\
protocol_name = r"/24_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
# odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

for i in range(2):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(19):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.idle(); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(rep_dead_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.idle(); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(3):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(off_time)

# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "dark", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "pun", "dark", "pun", "dark"
                         ], \
                durations=[odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, 60, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, 60, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           pun_on_time, off_time, pun_on_time, off_time
                           ]
                ) # 32.17 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_19580\1358382156.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_19580\1358382156.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_19580\1358382156.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input

Trace Conditioning Control 2 (Punishments Only) - 20 Training Trials, 10s Punishment Length

In [5]:
masks_path = r'D:\Data\OR83b_ppk72__20xpair_trace_subregion__pun_control\excitation_patterns\22_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\22_1__DAN_20241008_150329' # D:\Data\g12\
protocol_name = r"/22_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
# odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

for i in range(2):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(19):
    mirrors.idle(); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.idle(); time.sleep(odor_on_time); mirrors.idle(); time.sleep(rep_dead_time)

mirrors.idle(); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.idle(); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(3):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "dark", "dark", "pun", "dark", "dark", "dark", 
                         "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         ], \
                durations=[odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, 60, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, 60, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           ]
                ) # 30.8 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_7096\2107737790.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_7096\2107737790.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_7096\2107737790.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (an

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input

Learning Protocol - Trace Conditioning with Extinction (No Tests) - 20 Training Trials, 10 Extinction Trials

In [5]:
masks_path = r'D:\Data\OR83b_ppk72_20xpair_trace_extinction\excitation_patterns\2_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\2_1__DAN_20240716_151540' # D:\Data\g12\
protocol_name = r"/2_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

extinction_off_time = train_off_time + pun_on_time + train_off_time

for i in range(20):
    print("Train iteration: " + str(i))

    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(rep_dead_time)

for i in range(10):
    print("Extinction iteration: " + str(i))

    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(extinction_off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(rep_dead_time)


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         ], \
                durations=[odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           ]
                ) # 31 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_21724\2502799372.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_21724\2502799372.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_21724\2502799372.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior 

Train iteration: 0
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 1
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 2
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 3
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 4
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 5
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 6
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 7
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 8
DMD ON,  inpu

Learning Protocol - Trace Conditioning with Extinction (With Turning off Illuminators) - 20 Training Trials, 10 Extinction Trials

In [5]:
masks_path = r'D:\Data\OR83b_ppk72_20xpair_trace_extinction\excitation_patterns\1_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\1_1__DAN_20240716_120920' # D:\Data\g12\
protocol_name = r"/1_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

extinction_off_time = train_off_time + pun_on_time + train_off_time

for i in range(2):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); 

print("Turn off illumination laser")

time.sleep(60)

for i in range(20):
    print("Train iteration: " + str(i))
    if (i==14):
        print("Turn on illumination laser")

    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(rep_dead_time)

for i in range(10):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(extinction_off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(rep_dead_time)


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", 
                         ], \
                durations=[odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, 60, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, extinction_off_time, odor_on_time, rep_dead_time, 
                           ]
                ) # 36 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_23360\2588761344.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_23360\2588761344.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_23360\2588761344.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Turn off illumination laser
Train iteration: 0
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 1
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 2
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 3
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 4
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 5
DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

Train iteration: 6
DMD ON,  input voltage 

Time Calculation

In [1]:
on_time = 15
on_time_split = 10
odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

extinction_off_time = train_off_time + pun_on_time + train_off_time

durations = [odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, 60, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, 60, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           pun_on_time, off_time, pun_on_time, off_time
                           ]

t = 0
for i in durations:
    t+= i


print(t/60)

32.166666666666664
